# On essaie d'ouvrir les CSV fournis par Eric

Consigne : 

> trouver la variable "CYCLE" et définir la donnée correspondante à la date du 13/10/2014 13:09 par exemple

In [57]:
import csv 
from dateutil.parser import parse

fname = "./jour3/RechercheCSV.csv"

def transpose(values: dict) -> list:
    res = []
    transposed = list(zip(*values.values(), strict=True))
    for item in transposed:
        res.append(dict(zip(values.keys(), item, strict=True)))
    return res

def convert_types(values):
    for value in values:
        value["DATE"] = parse(value["DATE"])

def ouverture_avec_csv():
    res = {}
    with open(fname) as f_csv:
        csv_handler = csv.reader(f_csv, delimiter=";")
        for line in csv_handler:
            if "[ANALYSES]" in line:
                continue
            if not line:
                continue
            header, *values = line
            res[header] = values
            
    return res

In [58]:
res = ouverture_avec_csv()
res_transpose = transpose(res)
res_converted = convert_types(res_transpose)

In [56]:
for data in res_transpose:
    print(data)

{'DATE': '15/09/14 11:49:41', 'ACQUISITION-STARTDATE': '15/09/14 11:49:41', 'ACQUISITION-ENDDATE': '15/09/14 12:09:51', 'STATUS': 'DIAGNOSIS', 'SETUP_NAME': 'DEFAULT', 'CYCLE': 'TRI133', 'TEST-LABEL': '011'}
{'DATE': '01/10/14 16:10:02', 'ACQUISITION-STARTDATE': '01/10/14 16:10:02', 'ACQUISITION-ENDDATE': '01/10/14 16:30:13', 'STATUS': 'DIAGNOSIS', 'SETUP_NAME': 'DEFAULT', 'CYCLE': 'TRI133', 'TEST-LABEL': '012'}
{'DATE': '06/10/14 16:31:56', 'ACQUISITION-STARTDATE': '06/10/14 16:31:56', 'ACQUISITION-ENDDATE': '06/10/14 16:52:07', 'STATUS': 'DIAGNOSIS', 'SETUP_NAME': 'DEFAULT', 'CYCLE': 'TRI133', 'TEST-LABEL': '013'}
{'DATE': '13/10/14 13:09:54', 'ACQUISITION-STARTDATE': '13/10/14 13:09:54', 'ACQUISITION-ENDDATE': '13/10/14 13:30:04', 'STATUS': 'DIAGNOSIS', 'SETUP_NAME': 'DEFAULT', 'CYCLE': 'TRI133', 'TEST-LABEL': '014'}
{'DATE': '20/10/14 12:04:48', 'ACQUISITION-STARTDATE': '20/10/14 12:04:48', 'ACQUISITION-ENDDATE': '20/10/14 12:24:58', 'STATUS': 'DIAGNOSIS', 'SETUP_NAME': 'DEFAULT', 

[{'DATE': '15/09/14 11:49:41',
  'ACQUISITION-STARTDATE': '15/09/14 11:49:41',
  'ACQUISITION-ENDDATE': '15/09/14 12:09:51',
  'STATUS': 'DIAGNOSIS',
  'SETUP_NAME': 'DEFAULT',
  'CYCLE': 'TRI133',
  'TEST-LABEL': '011'},
 {'DATE': '01/10/14 16:10:02',
  'ACQUISITION-STARTDATE': '01/10/14 16:10:02',
  'ACQUISITION-ENDDATE': '01/10/14 16:30:13',
  'STATUS': 'DIAGNOSIS',
  'SETUP_NAME': 'DEFAULT',
  'CYCLE': 'TRI133',
  'TEST-LABEL': '012'},
 {'DATE': '06/10/14 16:31:56',
  'ACQUISITION-STARTDATE': '06/10/14 16:31:56',
  'ACQUISITION-ENDDATE': '06/10/14 16:52:07',
  'STATUS': 'DIAGNOSIS',
  'SETUP_NAME': 'DEFAULT',
  'CYCLE': 'TRI133',
  'TEST-LABEL': '013'},
 {'DATE': '13/10/14 13:09:54',
  'ACQUISITION-STARTDATE': '13/10/14 13:09:54',
  'ACQUISITION-ENDDATE': '13/10/14 13:30:04',
  'STATUS': 'DIAGNOSIS',
  'SETUP_NAME': 'DEFAULT',
  'CYCLE': 'TRI133',
  'TEST-LABEL': '014'},
 {'DATE': '20/10/14 12:04:48',
  'ACQUISITION-STARTDATE': '20/10/14 12:04:48',
  'ACQUISITION-ENDDATE': '20/10/1